In [ ]:
. ../../../../scripts/nbs_header.ps1
. ../../../../scripts/core.ps1

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(GetExecutableSuffix) --execute-command "pwsh -c `"../../../../scripts/invoke-dib.ps1 test.dib`"" } | Invoke-Block -Retries 5

00:00:00 #1 [Debug] runWithTimeoutAsync / timeout: 60
00:00:00 #2 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:main@464-1020> }
00:00:00 #3 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral\temp\test
00:00:00 #4 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:00 #5 [Verbose] > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:00 #6 [Debug] runWithTimeoutAsync / timeout: 500
00:00:00 #7 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:01 #8 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:01 #9 [Verbose] awaitCompiler / Ping / result: Some(null) /

In [ ]:
{ . ../../../../apps/parser/dist/DibParser$(GetExecutableSuffix) test.dib spi } | Invoke-Block

00:00:00 #1 [Debug] writeDibCode / output: Spi / path: test.dib
00:00:00 #2 [Debug] parseDibCode / output: Spi / file: test.dib


In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(GetExecutableSuffix) --build-file test.spi test.fsx --timeout 10000 } | Invoke-Block

00:00:00 #1 [Debug] runWithTimeoutAsync / timeout: 60
00:00:00 #2 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:main@425-297> }
00:00:00 #3 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral\temp\test
00:00:00 #4 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:00 #5 [Verbose] > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:00 #6 [Debug] runWithTimeoutAsync / timeout: 500
00:00:00 #7 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:01 #8 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:01 #9 [Verbose] awaitCompiler / Ping / result: Some(null) / 

In [ ]:
dotnet fable --optimize --lang rs --extension .rs

Fable 4.14.0: F# to Rust compiler (status: alpha)

Thanks to the contributor! @fsoikin
Stand with Ukraine! https://standwithukraine.com.ua/

Parsing test.fsproj...
Retrieving project options from cache, in case of issues run `dotnet fable clean` or try `--noCache` option.
Project and references (1 source files) parsed in 189ms

Started Fable compilation...

Fable compilation finished in 1448ms

.\test.fsx(6,0): (6,2) warning FABLE: For Rust, support for F# static and module do bindings is disabled by default. It can be enabled with the 'static_do_bindings' feature. Use at your own risk!


In [ ]:
(Get-Content test.rs) `
    -replace [regex]::Escape("),);"), "));" `
| Set-Content test.rs

In [ ]:
cargo fmt --

In [ ]:
cargo build --release

   Compiling spiral_temp_test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
  --> apps\spiral\temp\test\./main.rs:11:27
   |
11 | use proptest::arbitrary::{any, Arbitrary};
   |                           ^^^
12 | use proptest::prelude::*;
   |     -------------------- the item `any` is already imported here
   |
   = note: `#[warn(unused_imports)]` on by default

  --> apps\spiral\temp\test\./main.rs:11:32
   |
11 | use proptest::arbitrary::{any, Arbitrary};
   |                                ^^^^^^^^^
12 | use proptest::prelude::*;
   |     -------------------- the item `Arbitrary` is already imported here

   --> apps\spiral\temp\test\./main.rs:190:15
    |
190 | use tracing::{info, Level};
    |               ^^^^

  --> apps\spiral\temp\test\./main.rs:46:8
   |
45 | impl Cart {
   | --------- associated items in this implementation
46 |     fn new() -> Cart {
   |        ^^^
...
50 |     fn add_item(&mut self, item: Item) {
   |        ^^^^^^^^
...
56 |     fn remove_item(&m

In [ ]:
$env:RUST_LOG="info"
{ cargo test --release } | Invoke-Block

   Compiling spiral_temp_test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
  --> apps\spiral\temp\test\./main.rs:11:27
   |
11 | use proptest::arbitrary::{any, Arbitrary};
   |                           ^^^
12 | use proptest::prelude::*;
   |     -------------------- the item `any` is already imported here
   |
   = note: `#[warn(unused_imports)]` on by default

  --> apps\spiral\temp\test\./main.rs:11:32
   |
11 | use proptest::arbitrary::{any, Arbitrary};
   |                                ^^^^^^^^^
12 | use proptest::prelude::*;
   |     -------------------- the item `Arbitrary` is already imported here

    Finished `release` profile [optimized] target(s) in 24.00s
     Running unittests main.rs (C:\home\git\polyglot\target\release\deps\spiral_temp_test-0f3f76a0f233bf3e.exe)

running 3 tests
test test_parse_number ... ok
2024-03-28T21:53:56.119327Z  INFO spiral_temp_test: input=Identifier("T1dA27Qf2MI03O8f")
2024-03-28T21:53:56.119434Z  INFO spiral_temp_test: input=Identifi

In [ ]:
{ . $ScriptDir/../../../../target/release/spiral_temp_test$(GetExecutableSuffix) } | Invoke-Block

app=test
